In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
ROOT = '/media/nvme/Datasets/Pneumothorax/NIH/Images'

In [3]:
CSV_PATH = '/media/nvme/Datasets/Pneumothorax/NIH/Data_Entry_2017.csv'
nih_df = pd.read_csv(CSV_PATH)
print(nih_df.shape)
print(nih_df.columns)
nih_df.tail()

(112120, 12)
Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11'],
      dtype='object')


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN
112119,00030805_000.png,No Finding,0,30805,27,M,PA,2048,2500,0.171,0.171,NaN


## Get ptx images

In [4]:
ptx_df = nih_df[nih_df['Finding Labels'] == 'Pneumothorax']
ptx_df = ptx_df.rename(columns={'Image Index':'NIH_ID'})
print(ptx_df.shape)
ptx_df.tail()

(2194, 12)


,NIH_ID,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
111812,00030636_009.png,Pneumothorax,9,30636,45,M,AP,3056,2544,0.139000,0.139000,NaN
111813,00030636_010.png,Pneumothorax,10,30636,46,M,PA,2021,2021,0.194311,0.194311,NaN
111839,00030637_010.png,Pneumothorax,10,30637,49,M,PA,2021,2021,0.194311,0.194311,NaN
111841,00030637_012.png,Pneumothorax,12,30637,49,M,PA,2021,2021,0.194311,0.194311,NaN
112040,00030753_000.png,Pneumothorax,0,30753,54,F,AP,3056,2544,0.139000,0.139000,NaN


## Create Path

In [5]:
ptx_df.loc[:, 'Path'] = [os.path.join(ROOT, id) for id in ptx_df['NIH_ID']]

## Map Id to SIIM

In [6]:
conversion_csv = '/media/nvme/Datasets/Pneumothorax/NIH/kaggle_to_nih_id.csv'
conversion_df = pd.read_csv(conversion_csv).set_index('NIH_ID')
print(conversion_df.shape)
conversion_df.tail()

(12047, 1)


,Kaggle_ID
NIH_ID,
00008182_001.png,1.2.276.0.7230010.3.1.4.8323329.6230.151787519...
00005778_006.png,1.2.276.0.7230010.3.1.4.8323329.6693.151787519...
00026480_008.png,1.2.276.0.7230010.3.1.4.8323329.6568.151787519...
00004832_002.png,1.2.276.0.7230010.3.1.4.8323329.6022.151787519...
00009925_016.png,1.2.276.0.7230010.3.1.4.8323329.642.1517875163...


In [7]:
ptx_df.loc[:, 'SIIM_ID'] = conversion_df.loc[ptx_df['NIH_ID'], 'Kaggle_ID'].reset_index().drop_duplicates('NIH_ID')['Kaggle_ID'].values
ptx_df.tail()

/home/eduardo/.conda/envs/deep/lib/python3.7/site-packages/pandas/core/indexing.py:1017: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


,NIH_ID,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Path,SIIM_ID
111812,00030636_009.png,Pneumothorax,9,30636,45,M,AP,3056,2544,0.139000,0.139000,NaN,/media/nvme/Datasets/Pneumothorax/NIH/Images/0...,1.2.276.0.7230010.3.1.4.8323329.13759.15178752...
111813,00030636_010.png,Pneumothorax,10,30636,46,M,PA,2021,2021,0.194311,0.194311,NaN,/media/nvme/Datasets/Pneumothorax/NIH/Images/0...,1.2.276.0.7230010.3.1.4.8323329.4181.151787518...
111839,00030637_010.png,Pneumothorax,10,30637,49,M,PA,2021,2021,0.194311,0.194311,NaN,/media/nvme/Datasets/Pneumothorax/NIH/Images/0...,1.2.276.0.7230010.3.1.4.8323329.13503.15178752...
111841,00030637_012.png,Pneumothorax,12,30637,49,M,PA,2021,2021,0.194311,0.194311,NaN,/media/nvme/Datasets/Pneumothorax/NIH/Images/0...,1.2.276.0.7230010.3.1.4.8323329.1616.151787516...
112040,00030753_000.png,Pneumothorax,0,30753,54,F,AP,3056,2544,0.139000,0.139000,NaN,/media/nvme/Datasets/Pneumothorax/NIH/Images/0...,1.2.276.0.7230010.3.1.4.8323329.10103.15178752...


In [9]:
ptx_df.rename(columns={'NIH_ID':'ImageId'}).to_csv('/media/nvme/Datasets/Pneumothorax/NIH/nih_ptx.csv', index=False)